# InjectX AI

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf


In [2]:
path=r"Data\datasetUSB.csv"

dataset=pd.read_csv(path)

dataset.head(5)

,Timestamp,Key,Interval_ms,Chars_per_Second,Error_Count,is_human
0,28/10/2024 18:46,[CMD (WINDOWS)],0,0,0,0
1,28/10/2024 18:46,r,1,1000,0,0
2,28/10/2024 18:46,p,"407,3","2,46",0,0
3,28/10/2024 18:46,o,17,"58,82",0,0
4,28/10/2024 18:46,w,15,"66,67",0,0


In [3]:
dataset.tail(5)

,Timestamp,Key,Interval_ms,Chars_per_Second,Error_Count,is_human
1095,28/10/2024 18:52,[BACKSPACE],"146,15","6,84",22,1
1096,28/10/2024 18:52,[BACKSPACE],141,"7,09",23,1
1097,28/10/2024 18:52,[BACKSPACE],"141,09","7,09",24,1
1098,28/10/2024 18:52,",","228,54","4,38",24,1
1099,28/10/2024 18:52,[SPACE],"113,5","8,81",24,1


In [4]:
dataset.shape


(1100, 6)

In [5]:
dataset['is_human'].value_counts()

is_human
1    600
0    500
Name: count, dtype: int64

In [6]:
dataset

,Timestamp,Key,Interval_ms,Chars_per_Second,Error_Count,is_human
0,28/10/2024 18:46,[CMD (WINDOWS)],0,0,0,0
1,28/10/2024 18:46,r,1,1000,0,0
2,28/10/2024 18:46,p,"407,3","2,46",0,0
3,28/10/2024 18:46,o,17,"58,82",0,0
4,28/10/2024 18:46,w,15,"66,67",0,0
...,...,...,...,...,...,...
1095,28/10/2024 18:52,[BACKSPACE],"146,15","6,84",22,1
1096,28/10/2024 18:52,[BACKSPACE],141,"7,09",23,1
1097,28/10/2024 18:52,[BACKSPACE],"141,09","7,09",24,1
1098,28/10/2024 18:52,",","228,54","4,38",24,1


In [7]:
dataset.drop(axis=1,columns=['Timestamp','Key'],inplace=True)

dataset

,Interval_ms,Chars_per_Second,Error_Count,is_human
0,0,0,0,0
1,1,1000,0,0
2,"407,3","2,46",0,0
3,17,"58,82",0,0
4,15,"66,67",0,0
...,...,...,...,...
1095,"146,15","6,84",22,1
1096,141,"7,09",23,1
1097,"141,09","7,09",24,1
1098,"228,54","4,38",24,1


In [8]:

X_train = dataset[["Interval_ms", "Chars_per_Second", "Error_Count"]].values
y_train = dataset["is_human"].values  



## Neural Network using LSTM(Long Short-Term Memory) and cross-validation

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

dataset = dataset.replace(',', '.', regex=True).astype(float)

X_train = dataset[["Interval_ms", "Chars_per_Second", "Error_Count"]].values
y_train = dataset["is_human"].values  

window_size = 25
X_seq = []
y_seq = []

for i in range(len(X_train) - window_size + 1):
    X_seq.append(X_train[i:i + window_size])
    y_seq.append(y_train[i + window_size - 1])

X_seq = np.array(X_seq).astype(np.float32)
y_seq = np.array(y_seq).astype(np.float32)

kf = KFold(n_splits=5, shuffle=True, random_state=42)  # 5 folds

accuracy_scores = []

def build_model():
    model = Sequential([
        LSTM(50, input_shape=(X_seq.shape[1], X_seq.shape[2]), return_sequences=True),
        LSTM(50, return_sequences=True),
        LSTM(50),
        Dense(25, activation="relu"),
        Dense(10, activation="relu"),
        Dense(1, activation="sigmoid")
    ])
    model.compile(optimizer=Adam(), loss="binary_crossentropy", metrics=["accuracy"])
    return model

for train_index, val_index in kf.split(X_seq):
    X_train_fold, X_val_fold = X_seq[train_index], X_seq[val_index]
    y_train_fold, y_val_fold = y_seq[train_index], y_seq[val_index]
    
    model = build_model()
    model.fit(X_train_fold, y_train_fold, epochs=100, batch_size=10, validation_data=(X_val_fold, y_val_fold), verbose=0)
    
    val_loss, val_accuracy = model.evaluate(X_val_fold, y_val_fold, verbose=0)
    accuracy_scores.append(val_accuracy)
    print(f'Precision of the actual deploy: {val_accuracy:.4f}')

average_accuracy = np.mean(accuracy_scores)
print(f'\nPrecisión average of cross-validation: {average_accuracy:.4f}')


Precision of the actual deploy: 0.9954
Precision of the actual deploy: 1.0000
Precision of the actual deploy: 1.0000
Precision of the actual deploy: 0.9953
Precision of the actual deploy: 1.0000

Precisión average of cross-validation: 0.9981


## Why the precision is aprox 0.99?

At first glance you may think that the model is overtrained and overfitted by the high accuracy rate, however this is totally valid in this case because the determinant factor and the type of dataset we have show that the higher the Interval_ms variable is the conclusion that it is a human because of its write latency delay, on the other hand when it is low it represents that it is a script (malware).



In [11]:
model.save("Data/model.h5")

In [ ]:
from tensorflow.keras.models import load_model

window_size = 25

def predict_human_behavior(model, csv_path):
    dataset = pd.read_csv(csv_path)
    
    dataset = dataset.drop(columns=['Timestamp','Key'])
    dataset = dataset.replace(',', '.', regex=True).astype(float)
    
    X_data = dataset[["Interval_ms", "Chars_per_Second", "Error_Count"]].values
    
    X_seq = []
    for i in range(len(X_data) - window_size + 1):
        X_seq.append(X_data[i:i + window_size])
    
    X_seq = np.array(X_seq).astype(np.float32)
    
    predictions = model.predict(X_seq)
    mean_prediction = np.mean(predictions)
    
    result = "Human" if mean_prediction > 0.5 else "Not Human (you are being hacking dude)"
    
    print(f"Decision : {result}")

model = load_model("Data/model.h5")

predict_human_behavior(model, "Data\pruebasetNotHuman.csv")


1/1 [==============================] - 1s 572ms/step
Desicion: Not Human (you are being hacking dude)
